# SQL Problem Solutions for the 'Pedidos' Database

This notebook provides solutions to various SQL problems using the `Pedidos` database schema. The database represents a complex order management system with multiple interrelated tables that store data about products, employees, delivery personnel, orders, and final reports. Below is an overview of the database schema:

### **Database Schema Overview**
The `Pedidos` database consists of the following tables:

- **PRODUCTO**: Contains product details such as the product code, name, and price.
  - **Columns**:
    - `Codigo` (char(3)): Primary key representing the product code.
    - `Nombre` (VARCHAR(50)): Unique name of the product.
    - `Precio` (FLOAT(6,2)): Price of the product.

- **EMPLEADO**: Stores employee information, including name, social security number, work shift, and salary.
  - **Columns**:
    - `DNI` (CHAR(9)): Primary key representing the employee's identifier.
    - `Nombre` (VARCHAR(50)): Employee's name.
    - `Nss` (VARCHAR(11)): Social security number.
    - `Turno` (ENUM): Work shift, which can be 'morning', 'afternoon', or 'night'.
    - `Salario` (FLOAT(6,2)): Salary of the employee.

- **REPARTIDOR**: Stores delivery personnel details, similar to employees, but with an additional incentive attribute.
  - **Columns**:
    - `DNI` (CHAR(9)): Primary key representing the delivery person's identifier.
    - `Nombre` (VARCHAR(50)): Delivery person's name.
    - `Turno` (ENUM): Work shift ('morning', 'afternoon', 'night').
    - `Incentivo` (FLOAT(6,2)): Incentive for deliveries.

- **PEDIDO**: Represents an order, linking to employees and delivery personnel, and contains time information for tracking.
  - **Columns**:
    - `Numero` (CHAR(4)): Primary key representing the order number.
    - `Fecha` (DATE): Date when the order was placed.
    - `Importe` (FLOAT(6,2)): Amount of the order.
    - `DNI_ETM` (CHAR(9)): Foreign key linking to the employee who took the order.
    - `DNI_EP` (CHAR(9)): Foreign key linking to the employee who prepared the order.
    - `DNI_R` (CHAR(9)): Foreign key linking to the delivery person.
    - `Hora_tm`, `Hora_pre`, `Hora_rep` (TIME): Time attributes tracking different stages of order processing and delivery.

- **esta_compuesto**: Represents the relationship between products and their components.
  - **Columns**:
    - `Codigo_P` (CHAR(3)): Primary key linking to a product.
    - `Codigo_P_compuesto` (CHAR(3)): Primary key linking to a component of the product.

- **consta**: Represents the relationship between a product and an order, indicating how many units of a product are in a specific order.
  - **Columns**:
    - `Codigo_Pr` (CHAR(3)): Foreign key linking to a product.
    - `Numero_P` (CHAR(4)): Foreign key linking to an order number.
    - `cantidad` (TINYINT): Quantity of the product in the order.

- **PEDIDOS_FINALIZADOS**: Similar to the `PEDIDO` table but represents completed orders with additional details.
  - **Columns**:
    - Same as the `PEDIDO` table.
    - `Tiempo_transcurrido` (TIME): The time elapsed from order creation to completion.

- **RANKING_PRODUCTOS**: Tracks the ranking of products based on sales or other criteria.
  - **Columns**:
    - `Codigo` (CHAR(3)): Primary key representing the product code.
    - `Nombre` (VARCHAR(50)): Unique name of the product.
    - `Total` (INT): Total number representing the rank of the product.

### **Purpose of the Notebook**
This notebook is designed to solve a series of SQL problems using the schema described above. Each problem is presented with a detailed SQL query and, where applicable, relevant analysis or comments on the results.

The queries below illustrate various operations such as filtering, joining, aggregating, and subqueries. By following along with the notebook, you will gain insight into SQL techniques used to extract meaningful information from relational databases.

Let's get started with the problem solutions below.


1- Get the list of all products 

In [ ]:
%%sql
SELECT * FROM producto;



2- Get the order number, delivery person's ID, and delivery time for orders noted after 7 PM.

In [ ]:
%%sql
SELECT p.numero AS "N.Pedido", p.DNI_R , p.Hora_rep AS "Hora" FROM pedido p WHERE p.Hora_tm >= '19:00:00' and dni_r is not null;



3- Get all fields of employees earning between 900 and 1000 euros

In [ ]:
%%sql
SELECT * FROM empleado WHERE salario between 900 AND 1000;



4- Get the order number and amount for orders registered in November 2020 with an amount greater than 15 Euros.

In [ ]:
%%sql
SELECT numero, importe FROM pedido WHERE fecha LIKE '2020-11-%' AND Importe > 15; 



5- Get each delivery person's ID along with the number of orders delivered

In [ ]:
%%sql
SELECT p.DNI_R, count(p.DNI_R) as "Pedidos entregados" FROM pedido p 
INNER JOIN repartidor r ON p.DNI_R = r.DNI
group by p.DNI_R;



6- Get the number of orders completed each month (formatted with the month name instead of the number, e.g., November)

In [ ]:
%%sql
select date_format(p.fecha, '%M') as "month", count(p.numero) as "N. Pedidos" from pedido p
group by date_format(p.fecha, '%M'); -- Try grouping by year



7- Get a list with the name and ID of employees in the same field, with the ID preceding the name in the following format (e.g., 45776633P, Juan Rodríguez López) for employees with a "tarde" (afternoon) or "noche" (night) shift. Sort the list by ID.

In [ ]:
%%sql
select concat(dni,', ',nombre) AS 'P.Data', Turno FROM empleado where turno = 'tarde' or 'noche' order by dni;



8- Get the name, code, and price of products whose price is greater than or equal to the average price. Order from highest to lowest price.

In [ ]:
%%sql
select nombre, codigo, precio from producto where precio >= (select avg(precio) from producto) order by precio asc;



9- Get a list with the name and ID of employees who have never prepared any orders.

In [ ]:
%%sql
select e.nombre, e.DNI from empleado e where e.DNI not in (select dni_ETM from pedido);



10- Get the code, name, and price of products (last two in the same field) that are part of orders noted by "Luis" or "María Luisa". Order the list from highest to lowest value by order date.

In [ ]:
%%sql
select prod.codigo, concat(prod.nombre, prod.precio) as 'Producto' FROM producto prod, pedido p WHERE p.dni_etm = (select e.dni from empleado e where e.nombre = 'Luis' or 'Maria Luisa') order by p.fecha desc;



11- Get each delivery person's name, number of orders delivered, and average time it takes to deliver orders after they are prepared. Sort the list by average delivery time.

In [ ]:
%%sql
select r.nombre, count(p.numero) as 'Cantidad', avg(hora_rep) as 'Media H.' from repartidor r, pedido p group by r.nombre order by avg(hora_rep);



12- Get a list with the code, name, and price of the products that are the cheapest or most expensive of all. Sort the list by name.

In [ ]:
%%sql
select codigo, precio, nombre from producto where precio = (select min(precio) from producto) or precio = (select max(precio) from producto) order by nombre;



13- Get each product's name and code, along with the total number of orders in which it is included, provided that the total is greater than or equal to two. Order the list from highest to lowest number of orders.

In [ ]:
%%sql
SELECT PR.nombre, PR.codigo, count(P.Numero) as cantidad_pedidos 
FROM PEDIDO P, PRODUCTO PR, consta C
WHERE P.numero = C.numero_P
AND PR.codigo = C.codigo_Pr
GROUP BY PR.codigo
HAVING cantidad_pedidos>=2
ORDER BY cantidad_pedidos DESC;



14- Show a list of employees (name and NSS in the same column) who have noted any orders that include the product with code 13 and were delivered by the delivery person named 'Laura'.

In [ ]:
%%sql
SELECT concat(e.nombre, ', ', e.Nss) AS 'Empleados' FROM empleado e 
INNER JOIN pedido p ON e.DNI = p.DNI_ETM
INNER JOIN consta c ON p.Numero = c.Numero_P 
INNER JOIN repartidor r ON p.DNI_R =  r.DNI 
WHERE r.Nombre = 'Laura' AND c.Codigo_pr = '13';



15- Get the name of menu products along with the code of the products that compose them in orders from September 2020.

In [ ]:
%%sql
use pedidos;
select comp.Codigo_P_compuesto AS 'Nº', p.Nombre, group_concat(comp.Codigo_P) AS 'Ingredients' 
FROM esta_compuesto comp
JOIN producto p on comp.Codigo_P_compuesto = p.Codigo
JOIN consta on Codigo_Pr = p.Codigo
JOIN pedido on Numero_p = Numero
WHERE Fecha LIKE '2020-09-%' 
group by comp.Codigo_P_compuesto;



16- Insert the following data into the PEDIDO table, ensuring that only the necessary values are inserted into the corresponding fields.

In [ ]:
%%sql
use pedidos;
INSERT INTO pedido (Numero, Fecha, Importe, DNI_ETM, DNI_EP, DNI_R, Hora_tm, Hora_pre, Hora_rep)
VALUES ('0012', '2020-11-16', 20.00, (SELECT DNI FROM empleado WHERE DNI = '03232323P'), (SELECT DNI FROM empleado WHERE DNI = '04444444T'), NULL, '18:00:00', '18:03:00', NULL);
INSERT INTO pedido (Numero, Fecha, Importe, DNI_ETM, DNI_EP, DNI_R, Hora_tm, Hora_pre, Hora_rep)
VALUES ('0013', '2021-01-01', 67.00, (SELECT DNI FROM empleado WHERE DNI = '55555555L'), NULL, NULL, '14:00:00', NULL, NULL);
INSERT INTO pedido (Numero, Fecha, Importe, DNI_ETM, DNI_EP, DNI_R, Hora_tm, Hora_pre, Hora_rep)
VALUES ('0014', '2021-01-15', 13.00, (SELECT DNI FROM empleado WHERE DNI = '99999999X'), (SELECT DNI FROM empleado WHERE DNI = '55555555J'), (SELECT DNI FROM repartidor WHERE DNI = '04477744T'), '21:20:00', '21:30:00', '21:51:00');



17- Increase the amount of all orders by 10% for those completed in November 2020.

In [ ]:
%%sql
use pedidos;
UPDATE pedido
SET Importe = Importe * 1.10
WHERE Fecha LIKE '2020-11-%';



18- Delete orders with a delivery time of 25 minutes or more (from the time they were prepared) that were assigned to the delivery person named Alejandro Pardo López.

In [ ]:
%%sql
use pedidos;
DELETE FROM pedido
WHERE TIMESTAMPDIFF(MINUTE, Hora_pre, Hora_rep) >= 25
AND DNI_R = (SELECT DNI FROM repartidor WHERE nombre = 'Alejandro Pardo López');



19- Increase the incentive of delivery personnel by 50 euros for those who have delivered two or more orders.

In [ ]:
%%sql
use pedidos;
UPDATE repartidor 
SET Incentivo = Incentivo + 50.00
WHERE DNI IN (SELECT DNI_R FROM pedido GROUP BY DNI_R HAVING COUNT(*) >= 2);



20- Insert all delivered orders into the PEDIDOS_FINALIZADOS table, including, in addition to the fields of the orders table, the time elapsed from noting to delivery.

In [ ]:
%%sql
use pedidos;
INSERT INTO PEDIDOS_FINALIZADOS (Numero, Fecha, Importe, DNI_ETM, DNI_EP, DNI_R, Hora_tm, Hora_pre, Hora_rep, Tiempo_transcurrido)
SELECT Numero, Fecha, Importe, DNI_ETM, DNI_EP, DNI_R, Hora_tm, Hora_pre, Hora_rep, TIMEDIFF(Hora_rep, Hora_tm) AS Tiempo_transcurrido
FROM PEDIDO
WHERE Hora_rep IS NOT NULL;



21- Insert into the RANKING_PRODUCTOS table each product's code, name, and the total quantity ordered.

In [ ]:
%%sql
INSERT INTO RANKING_PRODUCTOS (Codigo, Nombre, Total)
SELECT p.Codigo, p.Nombre, SUM(c.cantidad) AS Total
FROM PRODUCTO p
JOIN consta c ON p.Codigo = c.Codigo_Pr
GROUP BY p.Codigo, p.Nombre;



22- Increase the salary of employees by 10 euros if they have noted any orders. Additionally, these employees must have also prepared an order.

In [ ]:
%%sql
use pedidos;
UPDATE empleado
SET salario = salario + 10
WHERE dni in (select DNI_etm from pedido) 
AND dni in (select DNI_ep from pedido);
